In [4]:
import sys
sys.path.insert(0, '../../Utilities/')

import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
        
q = 50
skip = 120

N0 = 199
N1 = 201
layers = [1, 50, 50, 50, 50, q]

data = scipy.io.loadmat('../Data/KdV.mat')

t_star = data['tt'].flatten()[:,None]
x_star = data['x'].flatten()[:,None]
Exact = np.real(data['uu'])

idx_t = 40

noise = 0.0    

idx_x = np.random.choice(Exact.shape[0], N0, replace=False)
x0 = x_star[idx_x,:]
u0 = Exact[idx_x,idx_t][:,None]
u0 = u0 + noise*np.std(u0)*np.random.randn(u0.shape[0], u0.shape[1])
    
idx_x = np.random.choice(Exact.shape[0], N1, replace=False)
x1 = x_star[idx_x,:]
u1 = Exact[idx_x,idx_t + skip][:,None]
u1 = u1 + noise*np.std(u1)*np.random.randn(u1.shape[0], u1.shape[1])

dt = (t_star[idx_t+skip] - t_star[idx_t]).item()
    
# Doman bounds
lb = x_star.min(0)
ub = x_star.max(0)

In [29]:
# Load IRK weights
q = 50
tmp = np.float32(np.loadtxt('../../Utilities/IRK_weights/Butcher_IRK%d.txt' % (q), ndmin = 2))
weights =  np.reshape(tmp[0:q**2+q], (q+1,q))     
IRK_alpha = weights[0:-1,:].T
IRK_beta = weights[-1:,:].T        
IRK_times = tmp[q**2+q:]
IRK_alpha

array([[ 7.27155653e-04,  1.57344667e-03,  1.39594928e-03, ...,
         1.45473145e-03,  1.45397626e-03,  1.45453354e-03],
       [-2.76877225e-04,  1.68994977e-03,  3.67410714e-03, ...,
         3.37842805e-03,  3.38107278e-03,  3.37912096e-03],
       [ 2.12501283e-04, -4.60933545e-04,  2.64763716e-03, ...,
         5.29816607e-03,  5.29296882e-03,  5.29680401e-03],
       ...,
       [-1.52996711e-06,  2.30557089e-06, -2.89193986e-06, ...,
         2.64763716e-03,  5.75620774e-03,  5.08277304e-03],
       [ 7.78573451e-07, -1.17325226e-06,  1.47161370e-06, ...,
        -2.94207450e-04,  1.68994977e-03,  3.65677685e-03],
       [-2.22312522e-07,  3.35006462e-07, -4.20195107e-07, ...,
         5.83620422e-05, -1.19135395e-04,  7.27155653e-04]], dtype=float32)

In [32]:
IRK_alpha = weights[0:-1,:]
IRK_weights = torch.from_numpy(np.reshape(tmp[0:q**2+q], (q+1, q))).to(torch.float64).T
IRK_weights.shape

torch.Size([50, 51])